In [1]:
!pip install langgraph langchain langchain-openai google-auth-oauthlib google-api-python-client langchain-google-genai


In [16]:
import os
import base64
from typing import TypedDict, Annotated, List, Literal
from email.mime.text import MIMEText

from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage
from langgraph.graph import StateGraph, END
from langgraph.checkpoint.memory import MemorySaver

# Google API Imports
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from googleapiclient.discovery import build



In [17]:
import os
os.listdir('/content')

['.config', 'sample_data']

In [18]:

GMAIL_SCOPES = ['https://www.googleapis.com/auth/gmail.modify']
def authorize_gmail():
    creds = None

    # Check if token already exists
    if os.path.isfile('token.json'):
        creds = Credentials.from_authorized_user_file(
            'token.json', GMAIL_SCOPES
        )

    # If no valid credentials, start manual OAuth
    if not creds or not creds.valid:
        oauth_flow = InstalledAppFlow.from_client_secrets_file(
            'credentials.json',
            scopes=GMAIL_SCOPES,
            redirect_uri='urn:ietf:wg:oauth:2.0:oob'
        )

        auth_link, _ = oauth_flow.authorization_url(
            access_type='offline',
            include_granted_scopes='true'
        )

        print("\n OPEN THIS LINK IN YOUR BROWSER:\n")
        print(auth_link)

        auth_code = input("\n Enter the authorization code here: ").strip()

        oauth_flow.fetch_token(code=auth_code)
        creds = oauth_flow.credentials

        # Save token
        with open('token.json', 'w') as f:
            f.write(creds.to_json())

    return build('gmail', 'v1', credentials=creds)
    service = get_gmail_service()

In [21]:
gmail_service = authorize_gmail()
print("Gmail authorization successful")



 OPEN THIS LINK IN YOUR BROWSER:

https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=1077630780818-mt5uinqlt0v0dktbl4eb7m34tg858kjv.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fgmail.modify&state=poFkrsCphq4trm4WngXa4o7eluDCkg&access_type=offline&include_granted_scopes=true

 Enter the authorization code here: 4/1ASc3gC2N_C2RTh9Hx9bY19NPGEkanmoa-Lp1OX2qYxLg5VNZi9xELoSBL-Q
Gmail authorization successful


In [22]:
def read_inbox(service):
    results = service.users().messages().list(
        userId='me',
        labelIds=['INBOX'],
        maxResults=5
    ).execute()

    messages = results.get('messages', [])

    if not messages:
        print("No messages found.")
        return

    for msg in messages:
        msg_data = service.users().messages().get(
            userId='me',
            id=msg['id'],
            format='metadata',
            metadataHeaders=['From', 'Subject']
        ).execute()

        headers = msg_data['payload']['headers']
        email_data = {}

        for h in headers:
            email_data[h['name']] = h['value']

        print("\n📧 New Email")
        print("From:", email_data.get('From'))
        print("Subject:", email_data.get('Subject'))

In [23]:
service = authorize_gmail()
print("Gmail service ready")

Gmail service ready


In [24]:
read_inbox(service)


📧 New Email
From: Google <no-reply@accounts.google.com>
Subject: Security alert

📧 New Email
From: "Udemy Instructor: TechLynk Selenium | DevOps | GenAI | Cloud" <no-reply@e.udemymail.com>
Subject: 🌟 Upgrade Your Skills — Udemy Courses (Up to 90% Off!)

📧 New Email
From: lisachristinevalaparla <notifications@github.com>
Subject: Re: [lisachristinevalaparla/ChatApp] pages commit (PR #7)

📧 New Email
From: "Udemy Instructor: Horizon Tech" <no-reply@e.udemymail.com>
Subject: inhorizontech@gmail.com

📧 New Email
From: Hindustan Copper Recruitment <noreply@jobalertshub.com>
Subject: Hindustan Copper Young Professional Job Notification


In [2]:
from openai import OpenAI

client = OpenAI(api_key="OPENAI_API_KEY")
for model in client.models.list():
    print(model.id)


gpt-3.5-turbo
gpt-5.2-codex
gpt-4o-mini-tts-2025-12-15
gpt-realtime-mini-2025-12-15
gpt-audio-mini-2025-12-15
chatgpt-image-latest
davinci-002
babbage-002
gpt-3.5-turbo-instruct
gpt-3.5-turbo-instruct-0914
dall-e-3
dall-e-2
gpt-3.5-turbo-1106
tts-1-hd
tts-1-1106
tts-1-hd-1106
text-embedding-3-small
text-embedding-3-large
gpt-3.5-turbo-0125
gpt-4o
gpt-4o-2024-05-13
gpt-4o-mini-2024-07-18
gpt-4o-mini
gpt-4o-2024-08-06
gpt-4o-audio-preview
omni-moderation-latest
omni-moderation-2024-09-26
gpt-4o-audio-preview-2024-12-17
gpt-4o-mini-audio-preview-2024-12-17
o1-2024-12-17
o1
gpt-4o-mini-audio-preview
o3-mini
o3-mini-2025-01-31
gpt-4o-2024-11-20
gpt-4o-search-preview-2025-03-11
gpt-4o-search-preview
gpt-4o-mini-search-preview-2025-03-11
gpt-4o-mini-search-preview
gpt-4o-transcribe
gpt-4o-mini-transcribe
gpt-4o-mini-tts
o3-2025-04-16
o4-mini-2025-04-16
o3
o4-mini
gpt-4.1-2025-04-14
gpt-4.1
gpt-4.1-mini-2025-04-14
gpt-4.1-mini
gpt-4.1-nano-2025-04-14
gpt-4.1-nano
gpt-image-1
gpt-4o-audio-previ

In [26]:
! pip install langchain langgraph langchain-openai python-dotenv

In [17]:
from google import genai
from google.genai import types

client = genai.Client(
    api_key="GEMINI_API_KEY"
)

def llm_call(prompt: str) -> str:
    response = client.models.generate_content(
        model="gemini-2.5-flash",
        contents=prompt,
        config=types.GenerateContentConfig(
            response_mime_type="application/json"
        )
    )
    return response.text


In [33]:
import os
import re
import base64
from email.mime.text import MIMEText



In [34]:
from googleapiclient.discovery import build
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request

SCOPES = ["https://www.googleapis.com/auth/gmail.modify"]

def get_gmail_service():
    creds = None

    if os.path.exists("token.json"):
        creds = Credentials.from_authorized_user_file("token.json", SCOPES)

    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                "credentials.json", SCOPES
            )

            # Headless / Colab safe
            flow.redirect_uri = "urn:ietf:wg:oauth:2.0:oob"
            auth_url, _ = flow.authorization_url(prompt="consent")

            print("Go to this URL and authorize:")
            print(auth_url)

            code = input("Enter authorization code: ")
            flow.fetch_token(code=code)
            creds = flow.credentials

        with open("token.json", "w") as token:
            token.write(creds.to_json())

    return build("gmail", "v1", credentials=creds)


In [35]:
def fetch_recent_emails(service, max_results=10):
    results = service.users().messages().list(
        userId="me",
        labelIds=["INBOX", "UNREAD"],
        maxResults=max_results
    ).execute()

    return results.get("messages", [])


In [36]:
def read_inbox(service, msg_id):
    msg = service.users().messages().get(
        userId="me",
        id=msg_id,
        format="full"
    ).execute()

    headers = msg["payload"]["headers"]
    subject = sender = ""

    for h in headers:
        if h["name"] == "Subject":
            subject = h["value"]
        elif h["name"] == "From":
            sender = h["value"]

    body = ""
    payload = msg["payload"]

    if "parts" in payload:
        for part in payload["parts"]:
            if part["mimeType"] == "text/plain":
                data = part["body"].get("data")
                if data:
                    body = base64.urlsafe_b64decode(data).decode("utf-8", errors="ignore")
                    break
    else:
        data = payload["body"].get("data")
        if data:
            body = base64.urlsafe_b64decode(data).decode("utf-8", errors="ignore")

    return {
        "subject": subject,
        "from": sender,
        "body": body,
        "thread_id": msg["threadId"]
    }


In [37]:
def clean_email_body(body):
    body = re.sub(r"```[\s\S]*?```", "", body)
    body = re.split(r"\nOn .* wrote:", body)[0]
    body = "\n".join(
        line for line in body.splitlines()
        if not line.strip().startswith(">")
    )
    return body.strip()


In [38]:
def rule_based_triage(subject, body):
    text = (subject + " " + body).lower()

    ignore_keywords = [
        "job", "vacancy", "alert", "offer",
        "sale", "promotion", "udemy",
        "academy", "webinar", "github"
    ]

    if any(word in text for word in ignore_keywords):
        return "IGNORE"

    return "RESPOND"


In [39]:
def generate_reply(subject, body):
    return "Thank you for your email. I will get back to you shortly."


In [40]:
def create_draft_from_text(service, to, subject, reply, thread_id):
    message = MIMEText(reply)
    message["to"] = to
    message["subject"] = "Re: " + subject

    raw = base64.urlsafe_b64encode(message.as_bytes()).decode()

    service.users().drafts().create(
        userId="me",
        body={
            "message": {
                "raw": raw,
                "threadId": thread_id
            }
        }
    ).execute()


In [51]:
def ask_gmail(service, question):
    if "unread" in question.lower():
        results = service.users().messages().list(
            userId="me",
            labelIds=["INBOX", "UNREAD"]
        ).execute()

        count = len(results.get("messages", []))
        return f"You have {count} unread emails."

    return "Sorry, I can only answer questions about unread emails."


In [55]:
response = ask_gmail(gmail_service, "How many unread emails are there")
print(response)


You have 100 unread emails.


In [52]:
DEFAULT_MEETING_TIME = "2026-02-01 10:00 AM"

def process_all_unread_emails(service):
    messages = fetch_recent_emails(service, max_results=10)

    print(f"\n📬 Total unread emails: {len(messages)}\n")

    if not messages:
        print("🎉 No unread emails.")
        return

    for i, msg in enumerate(messages, 1):
        print("=" * 70)

        email_data = read_inbox(service, msg["id"])
        subject = email_data["subject"]
        sender = email_data["from"]
        body = clean_email_body(email_data["body"])
        thread_id = email_data["thread_id"]

        # ---- BRIEF BODY ----
        brief_body = body[:200].replace("\n", " ")
        if len(body) > 200:
            brief_body += "..."

        print(f"📧 Email {i}")
        print(f"From    : {sender}")
        print(f"Subject : {subject}")
        print("Body    :")
        print(brief_body if brief_body else "(No content)")

        decision = rule_based_triage(subject, body)

        if decision == "IGNORE":
            print("\n⏭️ No action required")
            continue

        # ---- DRAFT ----
        reply = generate_reply(subject, body)

        print("\n---------------- DRAFT REPLY ----------------")
        print(reply)
        print("--------------------------------------------")

        create_draft_from_text(
            service,
            sender,
            subject,
            reply,
            thread_id
        )

        print("\n📝 Draft created")

        # ---- CALENDAR DISPLAY ----
        meeting_keywords = ["meeting", "call", "discussion", "interview", "invite"]

        if any(word in (subject + body).lower() for word in meeting_keywords):
            print("📅 Calendar Event : CREATED")
            print(f"⏰ Time           : {DEFAULT_MEETING_TIME}")
        else:
            print("📅 Calendar Event : Not required")

    print("\n✅ Done processing all unread emails")


In [54]:
gmail_service = get_gmail_service()
process_all_unread_emails(gmail_service)



📬 Total unread emails: 10

📧 Email 1
From    : "Dr. Venkata Raju" <invitations@linkedin.com>
Subject : I want to connect
Body    :
Dr. Venkata, Team Leader from OneSource is waiting for your response          Hi Bhavya, I’d like to join your professional network  Dr. Venkata Raju Team leader, Qc bioassay, onesource speciality pha...

---------------- DRAFT REPLY ----------------
Thank you for your email. I will get back to you shortly.
--------------------------------------------

📝 Draft created
📅 Calendar Event : CREATED
⏰ Time           : 2026-02-01 10:00 AM
📧 Email 2
From    : onlinecourses@nptel.iitm.ac.in
Subject : Week 2 Feedback Form : The Joy of Computing using Python!!
Body    :
<div>Thank you for continuing with the course and hope you are enjoying it.</div><div><br></div><div>We would like to know if the expectations with which you joined this course are being met and hence...

---------------- DRAFT REPLY ----------------
Thank you for your email. I will get back to you s